In [101]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import pickle
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import pandas as pd
import numpy as np

def get_soup(url):
    page = requests.get(url).text
    return BeautifulSoup(page, "lxml")

In [102]:
anime_sales_df = pd.read_pickle("../data/anime_sales_df_v2.pickle")
studio_df = pd.read_pickle("../data/studio_df_v1.pickle")

In [148]:
# https://towardsdatascience.com/how-to-do-fuzzy-matching-in-python-pandas-dataframe-6ce3025834a6
# https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

def get_anime_link(sales_anime_name, studio_name):
    studio_anime_info = studio_df.loc[studio_name]["anime_info"]
    
    studio_anime_names = list(studio_anime_info.keys())
    studio_anime_links = list(studio_anime_info.values())
    
    ratios = list(map(lambda x: fuzz.ratio(sales_anime_name.lower(), x.lower()), studio_anime_names))
    
    matches = []
    
    max_index = np.argmax(ratios)
    if ratios[max_index] > 90:
        return studio_anime_links[max_index]
    return None
    

In [149]:
anime_sales_df["link"] = anime_sales_df.apply(lambda x: get_anime_link(x["title"], x["studio"]), axis=1)

In [168]:
cols = ["title", "episode", "broadcast", "genre", "duration", "rating",\
        "score", "member", "favorite"]

#single_cols = ["title", "episode", "score"]


first_el_cols = ["title", "rating", ""]

def parse_col_data(elem):
    text = elem.parent.find_all(text=True)
    
    text_clean = [tag.strip() for tag in text if len(tag.strip())\
                 and tag.strip() != ',']
    
    col_name = text_clean[0]
    data = text_clean[1:]

    col_name = col_name[:-1].lower()
    col_name = col_name[:-1] if col_name[-1] == "s" else col_name
        
    if col_name in cols:
        try:
            if col_name == "title":
                data = data[0]
            if col_name == "duration":
                data = data[0].split(" ")[0]
            if col_name == "episode":
                data = data[0]
            if col_name == "rating":
                data = data[0].split(" ")[0]
            if col_name == "member" or col_name == "favorite":
                data = int(data[0].replace(",",""))
            if col_name == "genre":
                data = data[::2]
            if col_name == "score":
                data = float(data[0])
            if col_name == "broadcast":
                match = re.compile(r'(\w+)s at (\d\d:\d\d)').match(data[0])
                if match:
                    data = match.groups()
                else:
                    data = None
        except:
            print("error:", col_name, data)
            data = None
    # If we don't need this column, return None, None
    if col_name not in cols:
        return None, None

    return col_name, data

def get_anime_data(anime_url):
    soup = get_soup(anime_url)

    info_els = soup.find_all(class_="dark_text")
    
    anime_dict = dict([parse_col_data(el) for el in info_els])
    
    # Remove any None entries
    if None in anime_dict:
        del anime_dict[None]
    
    return anime_dict

# get_anime_data("https://myanimelist.net/anime/37779/Yakusoku_no_Neverland")

In [174]:
count = 0

for name, link in anime_sales_df[["title", "link"]].values:
    if link:
        if name in anime_dicts and len(anime_dicts[name].keys()) > 7:
            continue
        
        count += 1
        if count % 5 == 0:
            time.sleep(1.5)
        if count % 17 == 0:
            time.sleep(5)
            
        data = get_anime_data(link)
        print(name)
        if len(data.keys()) < 7:
            print("error:", name, " ", link)
            time.sleep(20)
        anime_dicts[name] = data

Aa Megami-sama
A.I.C.O. Incarnation
Anime de Wakaru Shinryou Naika
Ansatsu Kyoushitsu 2nd Season: Kagaijugyou-hen
Koro-sensei Q!
Koyomimonogatari
Baki
Bakumatsu Kikansetsu Irohanihoheto
Big Order
Bishoujo Senshi Sailor Moon Crystal
Bonjour♪Koiaji Pâtisserie
Denpa Kyoushi
Devilman Crybaby
Dream Festival!
Dream Festival! R
Ebiten: Kouritsu Ebisugawa Koukou Tenmonbu
Emiya-san Chi no Kyou no Gohan
Hen Semi
Hetalia Axis Powers
Hetalia World Series
Hetalia The Beautiful World
Hetalia The World Twinkle
Houkago no Pleiades
IDOLiSH7 Vibrato
Puchimas! -Petit Idolmaster-
Jashin-chan Dropkick'
Junjou Romantica 3
Junketsu no Maria
Just Because!
Juubei-chan Siberia Yagyuu no Gyakushuu
Juuni Kokuki
Juuni Taisen
Juuou Mujin no Fafnir
Juushin Enbu -Hero Tales-
Juushinki Pandora
Kaijuu Girls: Ultra Kaijuu Gijinka Keikaku
Karneval
Kidou Shinsengumi Moeyo Ken
Kishin Houkou Demonbane
kiss x sis
Kuroko no Basuke 2nd Season
Kuroko no Basuke 3rd Season
Kuroshitsuji
Kuroshitsuji II
Kuroshitsuji Book of Circus


In [156]:
anime_sales_df[["title", "link"]]

,title,link
.hack//SIGN,.hack//SIGN,https://myanimelist.net/anime/48/hack__Sign
.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu,https://myanimelist.net/anime/298/hack__Tasoga...
.hack//Roots,.hack//Roots,https://myanimelist.net/anime/873/hack__Roots
009-1,009-1,https://myanimelist.net/anime/1583/009-1
07-Ghost,07-Ghost,https://myanimelist.net/anime/5525/07-Ghost
...,...,...
The Unlimited -Hyoubu Kyousuke-,The Unlimited -Hyoubu Kyousuke-,None
Zettai Shounen,Zettai Shounen,https://myanimelist.net/anime/308/Zettai_Shounen
Zipang,Zipang,https://myanimelist.net/anime/29/Zipang
Zombie-Loan,Zombie-Loan,https://myanimelist.net/anime/2404/Zombie-Loan


In [175]:
#with open('../data/clean_anime_dicts.pickle', 'wb') as to_write:
#    pickle.dump(anime_dicts, to_write)